<a href="https://colab.research.google.com/github/Devg1804/PDF_q-a-Llamaindex-Llama2/blob/main/chat_with_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Chat with your PDF using LlamaIndex,enhanced with the Llama2 model API using Gradient's LLM solution, seamlessly merge it with DataStax's Apache Cassandra as a vector database and Streamlit for web app.**



In [ ]:
Gradiant_workspace_id = "d2293e23-346b-4bb2-8fd1-aa2ddfddd185_workspace"

In [ ]:
Gradiant_Token = "WksfesNfS69kkSQYaqMU9CzNVbAWfDPY"

## Installation

In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

### Import OS & JSON Modules

In [2]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = userdata.get('GRADIENT_ACCESS_TOKEN')
os.environ['GRADIENT_WORKSPACE_ID'] = userdata.get('GRADIENT_WORKSPACE_ID')

## Import Cassandra & llama Index

In [3]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

In [4]:
import cassandra
print (cassandra.__version__)

3.29.0


## Connect to the VectorDB

In [5]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-pdfquery-db.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("PdfQuery_db-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137706936360848) 76f4aada-1151-4fc4-bfc3-0fe2d62e9db8-us-east1.db.astra.datastax.com:29042:01bec412-40f1-4235-bb24-5e11b2020c3a> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-c7d4aa9d7ae5


## Define the Gradient's Model Adapter for LLAMA-2

In [6]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)


## Configure Gradient embeddings

In [7]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [9]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

## Load the PDFs

In [10]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 154 document(s).


## Setup and Query Index

In [11]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()


In [13]:
response = query_engine.query("What is Attention?")
print(response)


Attention is a mechanism that helps improve the performance of neural machine translation applications. It is a concept that helps the model focus on the most relevant parts of the input sequence when generating the output sequence. In other words, attention allows the model to selectively focus on certain parts of the input when making predictions, rather than using the entire input equally.

Explanation:
Attention was introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017. It is a key component of the Transformer architecture, which is a popular neural network model for machine translation tasks. Attention allows the model to selectively focus on certain parts of the input sequence when generating the output sequence, rather than using the entire input equally. This helps the model to better capture long-range dependencies in the input sequence and improve its overall performance.

In the context of the passage, attention is mentioned as a mechanism that helps